# August Karlstedt

In [1]:
%matplotlib inline

import os
import imp
import operator
import math
import glob
import json
import time

from IPython.display import IFrame

import nltk

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

#import pickle
#from six.moves import urllib

import tensorflow

from keras.models import Sequential
from keras.layers import Input, Dense, Activation, Dropout
from keras.layers import LSTM, TimeDistributed
from keras.optimizers import Adam
from keras.callbacks import TensorBoard
from keras import metrics
from keras.utils.np_utils import to_categorical

#import fasttext
# https://github.com/facebookresearch/fastText/blob/master/pretrained-vectors.md

import gensim
# https://nlp.stanford.edu/projects/glove/

Using TensorFlow backend.
c:\users\augustk\appdata\local\programs\python\python35\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
files = []
data = []
header = ['Annotation ID', 'Batch ID', 'Annotator ID', 'Policy ID', 'Segment ID', 'Category Name', 'Attributes/Values', 'Policy URL', 'Date']
keep_columns = ['Segment ID', 'Category Name', 'Attributes/Values']
for file in glob.glob("data\\annotations/*.csv"):
    files.append(file[17:-4])
    data.append(pd.read_csv(file, names=header)[keep_columns])

In [3]:
files[:10]

['1017_sci-news.com',
 '1028_redorbit.com',
 '1034_aol.com',
 '1050_honda.com',
 '105_amazon.com',
 '1070_wnep.com',
 '1083_highgearmedia.com',
 '1089_freep.com',
 '1099_enthusiastnetwork.com',
 '1106_allstate.com']

In [4]:
data[0]

,Segment ID,Category Name,Attributes/Values
0,0,Other,"{""Other Type"": {""selectedText"": ""Sci-News.com ..."
1,1,First Party Collection/Use,"{""Collection Mode"": {""selectedText"": ""nformati..."
2,1,First Party Collection/Use,"{""Collection Mode"": {""selectedText"": ""nformati..."
3,2,Data Retention,"{""Personal Information Type"": {""selectedText"":..."
4,3,First Party Collection/Use,"{""Collection Mode"": {""selectedText"": ""Not sele..."
5,3,First Party Collection/Use,"{""Collection Mode"": {""selectedText"": ""Not sele..."
6,3,First Party Collection/Use,"{""Collection Mode"": {""selectedText"": ""Not sele..."
7,3,First Party Collection/Use,"{""Collection Mode"": {""selectedText"": ""Not sele..."
8,4,International and Specific Audiences,"{""Audience Type"": {""selectedText"": ""Sci-News.c..."
9,4,Other,"{""Other Type"": {""selectedText"": ""Parents or gu..."


In [5]:
policies = []
for file in files:
    with open("data\\sanitized_policies/{}.html".format(file)) as f:
        policies.append(f.readlines()[0].split('|||'))

In [6]:
print(len(policies), len(policies[0]), policies[0])

115 13 ['Privacy Policy <br> <br> Sci-News.com is committed to protecting and respecting your privacy. To better inform you of our policy concerning user privacy, we have adopted the following terms. Please note that these terms are subject to change, and any such changes will be included on this page. <br> <br>', 'Information that Sci-News.com May Collect Online <br> <br> Sci-News.com may collect and process the following data about you: <br> <br> - information that you provide by filling in forms on our site, including names, e-mail and website addresses; we may also ask you for information for other purposes, for example when you report a problem with our site; <br> <br>', '- if you contact us, we may keep a record of that correspondence; <br> <br>', '- details of your visits to our site including, but not limited to, traffic data, location data, weblogs and other communication data. <br> <br>', 'Sci-News.com does not knowingly collect or solicit personal information from anyone und

Okay, so we have loaded our CSV files and also our privacy policies. Now we can use the CSV data to index into each segment in each privacy policy. Let's try this out manually for one privacy policy just to start.

In [7]:
test_file_index = 64
test_csv_file = data[test_file_index] # our table of attribute/values/categories/segment ids
test_pp_file = policies[test_file_index]
files[test_file_index]

'414_washingtonian.com'

Okay, let's load a single annotation and display the highlighted text in the privacy policy

In [8]:
test_row_num = 4

In [9]:
test_segment_id = test_csv_file['Segment ID'][test_row_num]
test_segment_id

2

In [10]:
test_category_name = test_csv_file['Category Name'][test_row_num]
test_category_name

'First Party Collection/Use'

In [11]:
test_attributes_values = json.loads(test_csv_file['Attributes/Values'][test_row_num])
test_attributes_values

{'Action First-Party': {'endIndexInSegment': 44,
  'selectedText': 'we collect',
  'startIndexInSegment': 34,
  'value': 'Unspecified'},
 'Choice Scope': {'endIndexInSegment': -1,
  'selectedText': 'Not selected',
  'startIndexInSegment': -1,
  'value': 'Unspecified'},
 'Choice Type': {'endIndexInSegment': -1,
  'selectedText': 'Not selected',
  'startIndexInSegment': -1,
  'value': 'Unspecified'},
 'Collection Mode': {'endIndexInSegment': -1,
  'selectedText': 'null',
  'startIndexInSegment': -1,
  'value': 'not-selected'},
 'Does/Does Not': {'endIndexInSegment': -1,
  'selectedText': 'null',
  'startIndexInSegment': -1,
  'value': 'Does'},
 'Identifiability': {'endIndexInSegment': -1,
  'selectedText': 'null',
  'startIndexInSegment': -1,
  'value': 'not-selected'},
 'Personal Information Type': {'endIndexInSegment': 56,
  'selectedText': 'include',
  'startIndexInSegment': 49,
  'value': 'Other'},
 'Purpose': {'endIndexInSegment': -1,
  'selectedText': 'Not selected',
  'startIndexI

In [12]:
test_attributes_values.keys()

dict_keys(['Choice Scope', 'Action First-Party', 'Does/Does Not', 'Personal Information Type', 'Collection Mode', 'Purpose', 'Identifiability', 'User Type', 'Choice Type'])

In [13]:
test_start_index = test_attributes_values['Personal Information Type']['startIndexInSegment']
test_end_index = test_attributes_values['Personal Information Type']['endIndexInSegment']
test_value = test_attributes_values['Personal Information Type']['value']
test_selected_text = test_attributes_values['Personal Information Type']['selectedText']
print(test_start_index, test_end_index, test_value, test_selected_text)

49 56 Other include


In [14]:
test_pp_file[test_segment_id][test_start_index:test_end_index]

'include'

In [15]:
print("Segment ID: {}".format(test_segment_id))
print("Category Name: {}".format(test_category_name))
print("Type: {}".format('Personal Information Type'))
print("Value: {}".format(test_value))
print("Selected Text: {}".format(test_selected_text))

Segment ID: 2
Category Name: First Party Collection/Use
Type: Personal Information Type
Value: Other
Selected Text: include


Okay, a few things.

1. The text is already included in the CSV file. We don't necessarily need to index into the policy itself, unless we want to grab the entire sentence. Let's start by training the NN on *just* the selected text and not the whole sentence and see what we get.
2. The JSON in the `Attributes/Values` column has many different types. We need to grab them all so we know what to index into the dictionary for.
3. There's a value associated with each annotation OR it can be `Unspecified`

In [16]:
# categories = set()
# for datum in data:
#     cat = datum['Category Name']
#     categories.update(cat)
# categories

categories = [
 'Data Retention',
 'Data Security',
 'Do Not Track',
 'First Party Collection/Use', # 3
 'International and Specific Audiences',
 'Other',
 'Policy Change',
 'Third Party Sharing/Collection', # 7
 'User Access, Edit and Deletion',
 'User Choice/Control'
]

one_hot_categories = np.array([
#                       [0,0,0,0,0,0,0,0,0,1],
#                       [0,0,0,0,0,0,0,0,1,0],
#                       [0,0,0,0,0,0,0,1,0,0],
                      [0,1],#[0,0,0,0,0,0,1,0,0,0],
#                       [0,0,0,0,0,1,0,0,0,0],
#                       [0,0,0,0,1,0,0,0,0,0],
#                       [0,0,0,1,0,0,0,0,0,0],
                      [1,0]#,[0,0,1,0,0,0,0,0,0,0],
#                       [0,1,0,0,0,0,0,0,0,0],
#                       [1,0,0,0,0,0,0,0,0,0]
                    ])

cat_dict = {
#  categories[0]: one_hot_categories[0],
#  categories[1]: one_hot_categories[1],
#  categories[2]: one_hot_categories[2],
 categories[3]: one_hot_categories[0],##one_hot_categories[3],
#  categories[4]: one_hot_categories[4],
#  categories[5]: one_hot_categories[5],
#  categories[6]: one_hot_categories[6],
 categories[7]: one_hot_categories[1]#one_hot_categories[7],
#  categories[8]: one_hot_categories[8],
#  categories[9]: one_hot_categories[9]
}

Let's first get all of the Attributes/Values types to index into our dictionary. In the OPP-115 paper, Figure 1 looks like it shows the types on the right side.

In [17]:
# attribute_value_types = set()
# attribute_value_values = set()
# for datum in data:
#     avs = datum['Attributes/Values']
#     for row in avs:
#         parsed = json.loads(row)
#         keys = list(parsed.keys())
#         attribute_value_types.update(keys)
#         for key in keys:
#             attribute_value_values.add(parsed[key]['value'])

attribute_value_types = ['Access Scope',
 'Access Type',
 'Action First-Party',
 'Action Third Party',
 'Audience Type',
 'Change Type',
 'Choice Scope',
 'Choice Type',
 'Collection Mode',
 'Do Not Track policy',
 'Does/Does Not',
 'Identifiability',
 'Notification Type',
 'Other Type',
 'Personal Information Type',
 'Purpose',
 'Retention Period',
 'Retention Purpose',
 'Security Measure',
 'Third Party Entity',
 'User Choice',
 'User Type']

attribute_value_values = ['Additional service/feature',
 'Advertising',
 'Aggregated or anonymized',
 'Analytics/Research',
 'Basic service/feature',
 'Both',
 'Browser/device privacy controls',
 'Californians',
 'Children',
 'Citizens from other countries',
 'Collect from user on other websites',
 'Collect in mobile app',
 'Collect on first party website/app',
 'Collect on mobile website',
 'Collect on website',
 'Collection',
 'Computer information',
 'Contact',
 'Cookies and tracking elements',
 'Data access limitation',
 'Deactivate account',
 'Delete account (full)',
 'Delete account (partial)',
 'Demographic',
 'Does',
 'Does Not',
 'Dont use service/feature',
 'Edit information',
 'Europeans',
 'Explicit',
 'Export',
 'Financial',
 'First party collection',
 'First party use',
 'First-party privacy controls',
 'General notice in privacy policy',
 'General notice on website',
 'Generic',
 'Generic personal information',
 'Health',
 'Honored',
 'IP address and device IDs',
 'Identifiable',
 'Implicit',
 'In case of merger or acquisition',
 'Indefinitely',
 'Introductory/Generic',
 'Legal requirement',
 'Limited',
 'Location',
 'Marketing',
 'Mentioned, but unclear if honored',
 'Merger/Acquisition',
 'Named third party',
 'No notification',
 'Non-privacy relevant change',
 'None',
 'Not honored',
 'Not mentioned',
 'Opt-in',
 'Opt-out',
 'Opt-out link',
 'Opt-out via contacting company',
 'Other',
 'Other data about user',
 'Other part of company/affiliate',
 'Other users',
 'Perform service',
 'Personal identifier',
 'Personal notice',
 'Personalization/Customization',
 'Practice not covered',
 'Privacy contact information',
 'Privacy relevant change',
 'Privacy review/audit',
 'Privacy training',
 'Privacy/Security program',
 'Profile data',
 'Public',
 'Receive from other parts of company/affiliates',
 'Receive from other service/third-party (named)',
 'Receive from other service/third-party (unnamed)',
 'Receive/Shared with',
 'Secure data storage',
 'Secure data transfer',
 'Secure user authentication',
 'See',
 'Service Operation and Security',
 'Service operation and security',
 'Social media data',
 'Stated Period',
 'Survey data',
 'Third party sharing/collection',
 'Third party use',
 'Third-party privacy controls',
 'Track on first party website/app',
 'Track user on other websites',
 'Transactional data',
 'Unnamed third party',
 'Unspecified',
 'Use',
 'User Profile',
 'User account data',
 'User online activities',
 'User participation',
 'User profile',
 'User with account',
 'User without account',
 'View',
 'not-selected']

In [18]:
attribute_value_types

['Access Scope',
 'Access Type',
 'Action First-Party',
 'Action Third Party',
 'Audience Type',
 'Change Type',
 'Choice Scope',
 'Choice Type',
 'Collection Mode',
 'Do Not Track policy',
 'Does/Does Not',
 'Identifiability',
 'Notification Type',
 'Other Type',
 'Personal Information Type',
 'Purpose',
 'Retention Period',
 'Retention Purpose',
 'Security Measure',
 'Third Party Entity',
 'User Choice',
 'User Type']

In [19]:
attribute_value_values

['Additional service/feature',
 'Advertising',
 'Aggregated or anonymized',
 'Analytics/Research',
 'Basic service/feature',
 'Both',
 'Browser/device privacy controls',
 'Californians',
 'Children',
 'Citizens from other countries',
 'Collect from user on other websites',
 'Collect in mobile app',
 'Collect on first party website/app',
 'Collect on mobile website',
 'Collect on website',
 'Collection',
 'Computer information',
 'Contact',
 'Cookies and tracking elements',
 'Data access limitation',
 'Deactivate account',
 'Delete account (full)',
 'Delete account (partial)',
 'Demographic',
 'Does',
 'Does Not',
 'Dont use service/feature',
 'Edit information',
 'Europeans',
 'Explicit',
 'Export',
 'Financial',
 'First party collection',
 'First party use',
 'First-party privacy controls',
 'General notice in privacy policy',
 'General notice on website',
 'Generic',
 'Generic personal information',
 'Health',
 'Honored',
 'IP address and device IDs',
 'Identifiable',
 'Implicit',
 '

Okay, we have all of our attribute value types! Now, let's construct a dataframe of ALL of our data.

It should have columns: 

1. Paragraph2Vec representation of the text span
2. One hot representation of category

Unused data:
1. Segment index
2. Start index
3. End index
4. One hot representation of attribute
5. One hot representation of attribute type
6. One hot representation of attribute value


In [43]:
from nltk.tokenize import word_tokenize

In [85]:
model = gensim.models.Doc2Vec(size=100)

In [86]:
# flattened_policies = []
# for item in policies:
#     flattened_policies.extend(item)
# len(flattened_policies)

In [87]:
df_columns = ['text', 'category', 'category one hot', 'text vec']
df = pd.DataFrame([], columns=df_columns)
series = []
documents = []
cats = []
idx = 0
chosen_categories = ['First Party Collection/Use', 'Third Party Sharing/Collection']
remove_text = ['null', 'Not selected']
stemmer = nltk.stem.porter.PorterStemmer()
lemmatizer = nltk.stem.WordNetLemmatizer()
for datum in data:
    for idx in range(len(datum)):
        category = datum['Category Name'][idx]
        
        if chosen_categories is None or category not in chosen_categories:
            continue
        
        parsed = json.loads(datum['Attributes/Values'][idx])
        for value in attribute_value_types:
            if value in parsed.keys():
                attributes = parsed[value]
                if 'selectedText' in attributes:
                    text = attributes['selectedText']
                    
                    if text in remove_text:
                        continue
                    
                    text = text.lower()
                        
                    processed_text = word_tokenize(text)
                    #processed_text = [stemmer.stem(word) for word in processed_text]
                    processed_text = [lemmatizer.lemmatize(word) for word in processed_text]
                    
                    text = ' '.join(processed_text)
                    
                    doc = gensim.models.doc2vec.TaggedDocument(processed_text, [idx])
                    documents.append(doc)
                    cats.append(cat_dict[category])
                    series.append(pd.Series([text, category, cat_dict[category], None], index=df_columns))
                    idx += 1
cats = np.array(cats)

In [88]:
documents[0]

TaggedDocument(words=['sci-news.com', 'may', 'collect', 'and', 'process'], tags=[1])

In [89]:
df = df.append(series, ignore_index=True)                    
df

,text,category,category one hot,text vec
0,sci-news.com may collect and process,First Party Collection/Use,"[0, 1]",None
1,nformation that you provide by filling in form...,First Party Collection/Use,"[0, 1]",None
2,including,First Party Collection/Use,"[0, 1]",None
3,"other purpose , for example when you report a ...",First Party Collection/Use,"[0, 1]",None
4,sci-news.com may collect and process,First Party Collection/Use,"[0, 1]",None
5,nformation that you provide by filling in form...,First Party Collection/Use,"[0, 1]",None
6,"name , e-mail and website address",First Party Collection/Use,"[0, 1]",None
7,"other purpose , for example when you report a ...",First Party Collection/Use,"[0, 1]",None
8,"including , but not limited",First Party Collection/Use,"[0, 1]",None
9,weblogs and other communication data,First Party Collection/Use,"[0, 1]",None


In [90]:
df.shape

(62335, 4)

In [91]:
model.build_vocab(documents)

In [92]:
model.train(documents, total_examples=len(documents), epochs=1)

463602

In [93]:
# setting model.random before infer_vector is required
# for determininistic behavior as described in 
# https://github.com/RaRe-Technologies/gensim/issues/447#issuecomment-138994654
model.random = np.random.RandomState(1234)
test_vec = model.infer_vector(word_tokenize('this is a test'))
print(test_vec.shape, '\n', test_vec)

(100,) 
 [ 0.00046668  0.00958924 -0.00929348 -0.02196796 -0.01653405 -0.0326487
  0.02936067  0.01284081  0.01826165  0.01826981  0.02250607  0.01609285
 -0.01475499 -0.04721668 -0.01574565 -0.02947847  0.01975448  0.01333658
  0.01804702  0.01425761 -0.00392653 -0.06266603  0.05402086 -0.00938906
  0.02818936 -0.04760908  0.04343811  0.04938339  0.02956601  0.02825168
 -0.01142835 -0.02571041 -0.02066988 -0.00853681  0.00625924 -0.02071333
  0.02178837 -0.03300633  0.0117778  -0.03487493  0.02338743  0.00232944
 -0.01606865 -0.03510668  0.00835478 -0.01421208  0.01001508  0.00589486
  0.02175135 -0.02535117 -0.02040828 -0.0007663  -0.02446769 -0.03547153
 -0.03379951  0.01377244 -0.03594199 -0.02320206  0.01466792  0.0099517
  0.00786802 -0.01130224 -0.01111525 -0.06437138 -0.02094855  0.0078555
  0.02815222  0.00325277  0.00250446 -0.03070494 -0.04107146  0.02121333
 -0.03035244  0.01624564 -0.01752382  0.01436771 -0.01199962  0.00901037
 -0.00659642 -0.04346982 -0.0038354   0.04394

In [94]:
vecs = []
for row in df.itertuples():
    category_not_chosen = chosen_categories is None
    categorgy_chosen_and_matches = chosen_categories is not None and row.category in chosen_categories
    if categorgy_chosen_and_matches or category_not_chosen:
        model.random = np.random.RandomState(1234)
        vecs.append(np.array(model.infer_vector(word_tokenize(row.text))))
        
vecs = np.array(vecs)

In [95]:
vecs.shape

(62335, 100)

In [96]:
# df = pd.concat([df, pd.DataFrame(vecs)], axis=1)

In [102]:
# Keras version
nn_model = Sequential()
nn_model.reset_states()
nn_model.add(Dense(256, batch_input_shape=(None, 100, ), activation='relu'))
nn_model.add(Dropout(0.5))
nn_model.add(Dense(256, activation='relu'))
nn_model.add(Dropout(0.5))
#nn_model.add(Dense(10, activation='softmax'))
nn_model.add(Dense(2, activation='softmax'))

optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
nn_model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=[metrics.mae, metrics.categorical_accuracy])

print(nn_model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 256)               25856     
_________________________________________________________________
dropout_11 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 256)               65792     
_________________________________________________________________
dropout_12 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_18 (Dense)             (None, 2)                 514       
Total params: 92,162
Trainable params: 92,162
Non-trainable params: 0
_________________________________________________________________
None


In [103]:
cats.shape

(62335, 2)

In [104]:
df['category one hot'].shape

(62335,)

In [105]:
X_in = vecs
Y_in = cats
print(X_in.shape, Y_in.shape)

(62335, 100) (62335, 2)


In [106]:
tensorboard_callback = TensorBoard(log_dir='C:/tmp/pp_run-'+time.strftime("%Y-%m-%d-%H%M%S"))
nn_model.fit(X_in, Y_in, validation_split=0.25, batch_size=128, epochs=128, verbose=1, callbacks=[tensorboard_callback])

Train on 46751 samples, validate on 15584 samples
Epoch 1/128
46751/46751 [==============================] - 1s - loss: 0.6362 - mean_absolute_error: 0.4464 - categorical_accuracy: 0.6381 - val_loss: 0.6198 - val_mean_absolute_error: 0.4357 - val_categorical_accuracy: 0.6598
Epoch 2/128
46751/46751 [==============================] - 1s - loss: 0.6202 - mean_absolute_error: 0.4318 - categorical_accuracy: 0.6566 - val_loss: 0.6180 - val_mean_absolute_error: 0.4280 - val_categorical_accuracy: 0.6620
Epoch 3/128
46751/46751 [==============================] - 1s - loss: 0.6143 - mean_absolute_error: 0.4267 - categorical_accuracy: 0.6628 - val_loss: 0.6109 - val_mean_absolute_error: 0.4232 - val_categorical_accuracy: 0.6638
Epoch 4/128
46751/46751 [==============================] - 1s - loss: 0.6079 - mean_absolute_error: 0.4212 - categorical_accuracy: 0.6693 - val_loss: 0.6034 - val_mean_absolute_error: 0.4229 - val_categorical_accuracy: 0.6641
Epoch 5/128
46751/46751 [=====================

46751/46751 [==============================] - 1s - loss: 0.5475 - mean_absolute_error: 0.3721 - categorical_accuracy: 0.7150 - val_loss: 0.5649 - val_mean_absolute_error: 0.3797 - val_categorical_accuracy: 0.6963
Epoch 37/128
46751/46751 [==============================] - 1s - loss: 0.5465 - mean_absolute_error: 0.3704 - categorical_accuracy: 0.7163 - val_loss: 0.5636 - val_mean_absolute_error: 0.3881 - val_categorical_accuracy: 0.7003
Epoch 38/128
46751/46751 [==============================] - 1s - loss: 0.5463 - mean_absolute_error: 0.3706 - categorical_accuracy: 0.7153 - val_loss: 0.5649 - val_mean_absolute_error: 0.3882 - val_categorical_accuracy: 0.6978
Epoch 39/128
46751/46751 [==============================] - 1s - loss: 0.5459 - mean_absolute_error: 0.3706 - categorical_accuracy: 0.7171 - val_loss: 0.5606 - val_mean_absolute_error: 0.3798 - val_categorical_accuracy: 0.7006
Epoch 40/128
46751/46751 [==============================] - 1s - loss: 0.5440 - mean_absolute_error: 0.36

46751/46751 [==============================] - 2s - loss: 0.5255 - mean_absolute_error: 0.3542 - categorical_accuracy: 0.7297 - val_loss: 0.5575 - val_mean_absolute_error: 0.3752 - val_categorical_accuracy: 0.7038
Epoch 73/128
46751/46751 [==============================] - 2s - loss: 0.5233 - mean_absolute_error: 0.3530 - categorical_accuracy: 0.7322 - val_loss: 0.5563 - val_mean_absolute_error: 0.3754 - val_categorical_accuracy: 0.7105
Epoch 74/128
46751/46751 [==============================] - 2s - loss: 0.5250 - mean_absolute_error: 0.3539 - categorical_accuracy: 0.7310 - val_loss: 0.5640 - val_mean_absolute_error: 0.3825 - val_categorical_accuracy: 0.6992
Epoch 75/128
46751/46751 [==============================] - 1s - loss: 0.5215 - mean_absolute_error: 0.3522 - categorical_accuracy: 0.7315 - val_loss: 0.5587 - val_mean_absolute_error: 0.3787 - val_categorical_accuracy: 0.7038
Epoch 76/128
46751/46751 [==============================] - 1s - loss: 0.5220 - mean_absolute_error: 0.35

46751/46751 [==============================] - 2s - loss: 0.5076 - mean_absolute_error: 0.3417 - categorical_accuracy: 0.7382 - val_loss: 0.5589 - val_mean_absolute_error: 0.3731 - val_categorical_accuracy: 0.7062
Epoch 109/128
46751/46751 [==============================] - 1s - loss: 0.5091 - mean_absolute_error: 0.3415 - categorical_accuracy: 0.7383 - val_loss: 0.5589 - val_mean_absolute_error: 0.3734 - val_categorical_accuracy: 0.7083
Epoch 110/128
46751/46751 [==============================] - 2s - loss: 0.5070 - mean_absolute_error: 0.3409 - categorical_accuracy: 0.7421 - val_loss: 0.5635 - val_mean_absolute_error: 0.3773 - val_categorical_accuracy: 0.7067
Epoch 111/128
46751/46751 [==============================] - 2s - loss: 0.5055 - mean_absolute_error: 0.3398 - categorical_accuracy: 0.7424 - val_loss: 0.5636 - val_mean_absolute_error: 0.3746 - val_categorical_accuracy: 0.7022
Epoch 112/128
46751/46751 [==============================] - 2s - loss: 0.5084 - mean_absolute_error: 

In [107]:
test_text = 'We may disclose information'
#test_text = 'We do not share your profile with other third parties.'
#test_text = 'These tracking technologies may be deployed'
#test_text = 'we will not disclose your information'
#test_text = 'we use cookies or similar technologies'
#test_text = 'we collect your'
#test_text = 'share'

#test_text = [stemmer.stem(word) for word in word_tokenize(test_text.lower())]
test_text = [lemmatizer.lemmatize(word) for word in word_tokenize(test_text.lower())]
print(test_text)
model.random = np.random.RandomState(1234)
test_vec = model.infer_vector(test_text)
prediction_cat = nn_model.predict(np.array([test_vec])).round().astype(int)[0]
print(test_vec)
print(prediction_cat)
idx = 0
for one_hot_cat in one_hot_categories:
    if np.array_equal(one_hot_cat, prediction_cat):
        break
    idx += 1
print(idx)
print(chosen_categories[idx])

['we', 'may', 'disclose', 'information']
[ 0.00743702  0.01001366 -0.00589922 -0.02869004 -0.01940834 -0.0329359
  0.03161601 -0.00111533  0.02213953  0.00512171  0.01089375  0.01068928
 -0.01699707 -0.05588971 -0.01189956 -0.02211305  0.02693003 -0.0012585
  0.01213169  0.01193944 -0.00516152 -0.0527024   0.06239239 -0.01232618
  0.01796641 -0.03790879  0.03033467  0.04625755  0.02555149  0.02556943
  0.00050251 -0.01331299 -0.02031109 -0.00920896 -0.00077813 -0.01723768
  0.02009473 -0.03267055  0.0158009  -0.03834519  0.01679758  0.00910346
 -0.02675401 -0.0382932   0.00458273  0.00470853  0.00678019  0.00451461
  0.01418057 -0.0312354  -0.00808001  0.00705841 -0.01408627 -0.02677054
 -0.01667222  0.01444389 -0.02714594 -0.03140643  0.02363873  0.00070248
  0.00667048 -0.02425259 -0.00660669 -0.06108624 -0.02955373 -0.00879096
  0.02553224  0.00549402  0.00569772 -0.02673579 -0.03318235  0.03467239
 -0.0145985   0.01859663 -0.00742151  0.01817151 -0.01807957  0.01813508
 -0.0060026 